In [1]:
# now we will be using implicit discretization scheme

import numpy as np
from matplotlib import pyplot as py
%matplotlib inline
from scipy import linalg

In [2]:
def btcs(T0, nt, dt, delta, alpha, Tb):
    ny, nx = T0.shape
    M, N = nx - 2, ny - 2
    sigma = alpha * dt / delta**2
    A = lhs_operator(M, N, sigma)
    T = T0[1:-1, 1:-1].flatten()
    I, J = int(M/2), int(N /2)
    for n in range(nt):
        b = rhs_vector(T, M, N, sigma, Tb)
        T = linalg.solve(A, b)
        if T[J * M + I] >= 70.0:
            break
    print('Time step {}; temperature at center {}.'
          .format(n, T[J * M+I]))
    return map_1d_to_2d(T, nx, ny, Tb)
        

In [3]:
def map_1d_to_2d(T1, nx, ny, Tb):
    T = np.xeros((ny, nx))
    T[1:-1, 1:-1] = T1.reshape((ny-2, nx - 2))
    
    T[:, 0] = Tb
    T[0, :] = Tb
    T[:, -1] = T[:, -2]
    T[-1, :] = T[-2, :]
    return T

In [4]:
def rhs_vector(T, M, N, sigma, Tb):
    b = T / sigma
    for j in range(N):
        for i in range(M):
            I = j * M +i
            if i == 0:
                b[I] += Tb
            if j == 0:
                b[I] += Tb
            return b

In [5]:
def lhs_operator(M, N, sigma):
    A = np.zeros((M * N, M * N))
    for j in range(N):
        for i in range(M):
            I = j * M + 1
            IS = I - M
            IW = I - 1
            IE = I + 1
            IN = I + M
            if i == 0 and j == 0:
                A[I, I] = 1.0/sigma + 4.0
                A[I, IN] = -1.0
                A[I, IE] = -1.0
            elif i == 0 and j == N -1:
                A[I, I] = 1.0 / sigma + 3.0
                A[I, IS] = -1.0
                A[I, IE] = -1.0
            elif i == M-1 and j == 0:
                A[I, I] = 1.0 / sigma + 3.0
                A[I, IN] = -1.0
                A[I, IW] = -1.0
            elif i == M - 1 and j == N - 1:
                A[I, I] = 1.0 / sigma + 2.0
                A[I, IS] = -1.0
                A[I, IW] = -1.0
            elif i == 0:
                A[I, I] = 1.0 / sigma + 4.0
                A[I, IS] = -1.0
                A[I, IE] = -1.0
                A[I, IN] = -1.0
            elif i == M - 1:
                A[I,I] = 1.0 / sigma + 3.0
                A[I, IS] = -1.0
                A[I, IW] = -1.0
                A[I, IN] = -1.0
            elif j == N - 1:
                A[I, I] = 1.0 / sigma + 3.0
                A[I, IS] = -1.0
                A[I, IW] = -1.0
                A[I, IE] = -1.0
            elif j == 0:
                A[I, I] = 1.0 / sigma + 4.0
                A[I, IW] = -1.0
                A[I, IE] = -1.0
                A[I, IN] = -1.0
            else:
                A[I, I] = 1/sigma + 4.0
                A[I, IS] = -1.0
                A[I, IW] = -1.0
                A[I, IE] = -1.0
                A[I, IN] = -1.0
    return A

In [6]:
Lx = 0.01 # length of chip
Ly = 0.01 # width of chip
nx = 21
ny = 21

dx = Lx / (nx - 1)
dy = Ly / (ny - 1)
alpha = 1e-4 # thermal diffusivity

# now we have discretize our domain
x = np.linspace(0.0, Lx, num=nx)
y = np.linspace(0.0, Ly, num=ny)

# describe the initial conditions
T0 = 20.0 * np.ones((ny, nx))
T0[0, :] = 100.0
T0[:, 0] = 100.0

In [7]:
sigma = 0.5
dt = sigma / (alpha * (1 / dx**2 + 1 / dy**2))
nt = 500

In [8]:
T = btcs(T0, nt, dt, dx, alpha, 100.0)

LinAlgError: Matrix is singular.

In [ ]:
py.figure(figsize=(8.0, 5.0))
py.xlabel('x [m]')
py.ylabel('y [m]')
contf = py.contourf(x, y, T, 
                    levels= np.linspace(20.0, 100.0, num=51))

cbar = py.colorbar(contf)
cbar.set_label('Temperature [C]')